Since we donot have permission to save data in dbsf, using google drive to download the zip file of data then work on it

In [0]:
try:
    import gdown
    print("gdown is installed.")
except ImportError:
    print("gdown is not installed. Installing it now...")
    try:
        import subprocess
        subprocess.check_call(["pip", "install", "gdown"])
        print("gdown has been successfully installed.")
        import gdown  # Importing again to use it after installation
    except Exception as e:
        print(f"Error installing gdown: {e}")


from pyspark.ml.feature import VectorAssembler,VectorIndexer, OneHotEncoder,StringIndexer
from pyspark.sql.functions import when, col

import pandas as pd
import numpy as np

try:
    from xgboost.spark import SparkXGBClassifier
    print("sparkxgb is installed.")
except ImportError:
    print("sparkxgb is not installed. Installing it now...")
    try:
        import subprocess
        command = "pip install pyarrow pandas venv-pack xgboost"
        subprocess.run(command, shell=True, check=True)
        print("sparkxgb has been successfully installed.")
        from xgboost.spark import SparkXGBClassifier
    except Exception as e:
        print(f"Error installing sparkxgb: {e}")

from pyspark.ml.classification import LogisticRegression
from pyspark.sql.types import FloatType, IntegerType
from pyspark.ml.feature import VectorAssembler,VectorIndexer, OneHotEncoder,StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator



gdown is not installed. Installing it now...
gdown has been successfully installed.


sparkxgb is not installed. Installing it now...
sparkxgb has been successfully installed.


In [0]:
%%bash
#rm -r Trends_market_place_data
# Replace 'YOUR_FILE_ID' with the actual file ID from the Google Drive link
#file_id="1SznHDvix3NdpmiIloyDxeAjiFtY-nk38"

# Replace 'output_file.zip' with the desired output file name
#output_file="Trends_market_place_data.zip"

# Download the file from Google Drive using gdown
#gdown "https://drive.google.com/uc?id=${file_id}" -O "${output_file}"

#unzip Trends_market_place_data.zip
#rm -f Trends_market_place_data.zip
#rm -rf __MACOSX

In [0]:
# Creating spark dataframre from csv files:

# Define the folder path containing CSV files
folder_path = "file:/databricks/driver/Trends_market_place_data"

# Read all CSV files in the folder into a DataFrame
network_labelled_data = spark.read.option("header", "true").option("delimiter", "|").option("inferschema", "true").csv(folder_path + "/*.csv")

# Show the DataFrame
network_labelled_data.limit(5).toPandas()

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,label,detailed-label
0,1.525880e+09,CUmrqr4svHuSXJy5z7,192.168.100.103,51524.0,65.127.233.163,23.0,tcp,-,2.999051,0,...,-,0.0,S,3.0,180.0,0.0,0.0,-,Malicious,PartOfAHorizontalPortScan
1,1.525880e+09,CH98aB3s1kJeq6SFOc,192.168.100.103,56305.0,63.150.16.171,23.0,tcp,-,-,-,...,-,0.0,S,1.0,60.0,0.0,0.0,-,Malicious,PartOfAHorizontalPortScan
2,1.525880e+09,C3GBTkINvXNjVGtN5,192.168.100.103,41101.0,111.40.23.49,23.0,tcp,-,-,-,...,-,0.0,S,1.0,60.0,0.0,0.0,-,Malicious,PartOfAHorizontalPortScan
3,1.525880e+09,CDe43c1PtgynajGI6,192.168.100.103,60905.0,131.174.215.147,23.0,tcp,-,2.998796,0,...,-,0.0,S,3.0,180.0,0.0,0.0,-,Malicious,PartOfAHorizontalPortScan
4,1.525880e+09,CJaDcG3MZzvf1YVYI4,192.168.100.103,44301.0,91.42.47.63,23.0,tcp,-,-,-,...,-,0.0,S,1.0,60.0,0.0,0.0,-,Malicious,PartOfAHorizontalPortScan


In [0]:
#Processing data
columns_to_remove = [0, 1, 2, 4, 12, 13, 14, 20, 22]
columns_to_drop = [network_labelled_data.columns[i] for i in columns_to_remove]

# Drop the specified columns
df_without_columns = network_labelled_data.drop(*columns_to_drop)

# Define the transformation using when
df_without_columns = df_without_columns.withColumn(
    "Malicious", 
    when(col("label") == "Benign", 0).otherwise(1)
)

# Drop the original label column if needed
df_without_columns = df_without_columns.drop("label")

# Columns to transform
columns_to_transform = [3, 4, 5, 6, 8]
columns_to_transform = [df_without_columns.columns[i] for i in columns_to_transform]

# Columns to put 0 if '-'
zeros = [4, 5, 6]
zeros = [df_without_columns.columns[i] for i in zeros]

# Replace '-'
for column in zeros:
    df_without_columns = df_without_columns.withColumn(
        column,
        when((col(column) == "-") , 0).otherwise(col(column)),
    )

# Columns to convert to float
columns_to_convert_to_float = [4]
columns_to_convert_to_float = [df_without_columns.columns[i] for i in columns_to_convert_to_float]

for column in columns_to_convert_to_float:
    df_without_columns = df_without_columns.withColumn(column, col(column).cast(FloatType()))

# Columns to convert to int
column_mapping_convert = [0, 1]
column_mapping_convert = [df_without_columns.columns[i] for i in column_mapping_convert]

# Define the mapping for column renaming
column_mapping = {
    column_mapping_convert[0]: "id_orig_p",
    column_mapping_convert[1]: "id_resp_p",
}

# Rename columns using withColumnRenamed
for old_col, new_col in column_mapping.items():
    df_without_columns = df_without_columns.withColumnRenamed(old_col, new_col)


# Columns to convert to int
columns_to_convert_to_int = [0, 1, 5, 6, 9, 10, 11]
columns_to_convert_to_int = [df_without_columns.columns[i] for i in columns_to_convert_to_int]


# Remove rows where the first column contains an IP address
df_without_columns = df_without_columns.filter(~col(columns_to_convert_to_int[0]).rlike(r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}"))

for column in columns_to_convert_to_int:
    df_without_columns = df_without_columns.withColumn(column, col(column).cast(IntegerType()))


In [0]:
# Show the transformed DataFrame
df_without_columns.limit(5).toPandas()


,id_orig_p,id_resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,Malicious
0,59932,80,tcp,-,3.097754,0,0,S0,S,3,180,0,0.0,1
1,59932,80,tcp,-,0.000000,0,0,S0,S,1,60,0,0.0,1
2,59932,80,tcp,-,0.000000,0,0,S0,S,1,60,0,0.0,1
3,35883,53,udp,dns,5.005148,78,0,S0,D,2,134,0,0.0,0
4,43531,53,udp,dns,5.005145,78,0,S0,D,2,134,0,0.0,0


In [0]:
si = StringIndexer(inputCols=['proto', 'service', 'conn_state', 'history'],outputCols=['proto_idx','service_idx', 'conn_state_idx', 'history_idx'], handleInvalid = "keep")
ohe = OneHotEncoder(inputCols=['proto_idx','service_idx', 'conn_state_idx', 'history_idx'],outputCols=['proto_ohe','service_ohe', 'conn_state_ohe', 'history_ohe'])

#df_without_columns_si = si.fit(df_without_columns).transform(df_without_columns)
#df_without_columns_si.limit(5).display()

In [0]:
#df_without_columns_ohe = ohe.fit(df_without_columns_si).transform(df_without_columns_si)
#df_without_columns_ohe.limit(5).display()

In [0]:
va = VectorAssembler(inputCols=['proto_ohe','service_ohe', 'conn_state_ohe', 'history_ohe',
 'id_resp_p', 'id_orig_p', 'duration','orig_bytes', 'resp_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes' ],outputCol='features', handleInvalid = "skip")


lr = LogisticRegression(featuresCol='features',labelCol='Malicious')


xgb = SparkXGBClassifier(
  features_col="features",
  label_col="Malicious",
  num_workers=2
)



pl = Pipeline(stages=[si,ohe,va,xgb])

In [0]:
train, test = df_without_columns.randomSplit([0.005,.995], seed=0)

print("train size {} test size {}".format(train.count(),test.count()))



train size 24875782 test size 124818


In [0]:
model = pl.fit(train)

2023-12-02 15:17:16,573 INFO XGBoost-PySpark: _fit Running xgboost-2.0.2 on 2 workers with
	booster params: {'objective': 'binary:logistic', 'device': 'cpu', 'nthread': 1}
	train_call_kwargs_params: {'verbose_eval': True, 'num_boost_round': 100}
	dmatrix_kwargs: {'nthread': 1, 'missing': nan}
2023-12-02 15:21:35,489 INFO XGBoost-PySpark: _fit Finished xgboost training!


In [0]:
%%bash
rm -rf predictive_model
mkdir predictive_model

In [0]:
# Save the model to a folder
model_path = r"file:///databricks/driver/predictive_model"
model.write().overwrite().save(model_path)
print("model saved")

model saved


In [0]:
results = model.transform(test)
results.printSchema()

root
 |-- id_orig_p: integer (nullable = true)
 |-- id_resp_p: integer (nullable = true)
 |-- proto: string (nullable = true)
 |-- service: string (nullable = true)
 |-- duration: float (nullable = true)
 |-- orig_bytes: integer (nullable = true)
 |-- resp_bytes: integer (nullable = true)
 |-- conn_state: string (nullable = true)
 |-- history: string (nullable = true)
 |-- orig_pkts: integer (nullable = true)
 |-- orig_ip_bytes: integer (nullable = true)
 |-- resp_pkts: integer (nullable = true)
 |-- resp_ip_bytes: double (nullable = true)
 |-- Malicious: integer (nullable = false)
 |-- proto_idx: double (nullable = false)
 |-- service_idx: double (nullable = false)
 |-- conn_state_idx: double (nullable = false)
 |-- history_idx: double (nullable = false)
 |-- proto_ohe: vector (nullable = true)
 |-- service_ohe: vector (nullable = true)
 |-- conn_state_ohe: vector (nullable = true)
 |-- history_ohe: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: v

In [0]:
# function that calculate all five metrics
def evaluate(df):
    '''df is the test dataframe containing both the label, rawPrediction, and prediction columns. 
    This function obtains various model performance metrics'''
    evaluator_auc = BinaryClassificationEvaluator(labelCol='Malicious', rawPredictionCol='rawPrediction')
    AUC = evaluator_auc.evaluate(df)
    print("AUC = %g" % AUC)
    evaluator = MulticlassClassificationEvaluator(labelCol="Malicious", predictionCol="prediction", \
                                                  metricName="accuracy")
    accuracy = evaluator.evaluate(df)
    print("Accuracy = %g" % accuracy)

    evaluator.setMetricName("f1")
    
    f1 = evaluator.evaluate(df)
    print("f1 = %g" % f1)


In [0]:
evaluate(results)

AUC = 0.997349
Accuracy = 0.992641
f1 = 0.992657


In [0]:
results.limit(5).display()

id_orig_p,id_resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,Malicious,proto_idx,service_idx,conn_state_idx,history_idx,proto_ohe,service_ohe,conn_state_ohe,history_ohe,features,rawPrediction,prediction,probability
3,0,icmp,-,0.0,0,0,OTH,-,1,56,0,0.0,0,2.0,0.0,1.0,5.0,"Map(vectorType -> sparse, length -> 3, indices -> List(2), values -> List(1.0))","Map(vectorType -> sparse, length -> 5, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 8, indices -> List(1), values -> List(1.0))","Map(vectorType -> sparse, length -> 32, indices -> List(5), values -> List(1.0))","Map(vectorType -> sparse, length -> 57, indices -> List(2, 3, 9, 21, 49, 53, 54), values -> List(1.0, 1.0, 1.0, 1.0, 3.0, 1.0, 56.0))","Map(vectorType -> dense, length -> 2, values -> List(7.150057315826416, -7.150057315826416))",0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.9992157816886902, 7.842036429792643E-4))"
3,0,icmp,-,0.0,0,0,OTH,-,1,56,0,0.0,0,2.0,0.0,1.0,5.0,"Map(vectorType -> sparse, length -> 3, indices -> List(2), values -> List(1.0))","Map(vectorType -> sparse, length -> 5, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 8, indices -> List(1), values -> List(1.0))","Map(vectorType -> sparse, length -> 32, indices -> List(5), values -> List(1.0))","Map(vectorType -> sparse, length -> 57, indices -> List(2, 3, 9, 21, 49, 53, 54), values -> List(1.0, 1.0, 1.0, 1.0, 3.0, 1.0, 56.0))","Map(vectorType -> dense, length -> 2, values -> List(7.150057315826416, -7.150057315826416))",0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.9992157816886902, 7.842036429792643E-4))"
3,0,icmp,-,0.0,0,0,OTH,-,1,56,0,0.0,0,2.0,0.0,1.0,5.0,"Map(vectorType -> sparse, length -> 3, indices -> List(2), values -> List(1.0))","Map(vectorType -> sparse, length -> 5, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 8, indices -> List(1), values -> List(1.0))","Map(vectorType -> sparse, length -> 32, indices -> List(5), values -> List(1.0))","Map(vectorType -> sparse, length -> 57, indices -> List(2, 3, 9, 21, 49, 53, 54), values -> List(1.0, 1.0, 1.0, 1.0, 3.0, 1.0, 56.0))","Map(vectorType -> dense, length -> 2, values -> List(7.150057315826416, -7.150057315826416))",0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.9992157816886902, 7.842036429792643E-4))"
3,0,icmp,-,0.0,0,0,OTH,-,1,56,0,0.0,0,2.0,0.0,1.0,5.0,"Map(vectorType -> sparse, length -> 3, indices -> List(2), values -> List(1.0))","Map(vectorType -> sparse, length -> 5, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 8, indices -> List(1), values -> List(1.0))","Map(vectorType -> sparse, length -> 32, indices -> List(5), values -> List(1.0))","Map(vectorType -> sparse, length -> 57, indices -> List(2, 3, 9, 21, 49, 53, 54), values -> List(1.0, 1.0, 1.0, 1.0, 3.0, 1.0, 56.0))","Map(vectorType -> dense, length -> 2, values -> List(7.150057315826416, -7.150057315826416))",0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.9992157816886902, 7.842036429792643E-4))"
3,0,icmp,-,0.0,0,0,OTH,-,1,56,0,0.0,0,2.0,0.0,1.0,5.0,"Map(vectorType -> sparse, length -> 3, indices -> List(2), values -> List(1.0))","Map(vectorType -> sparse, length -> 5, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 8, indices -> List(1), values -> List(1.0))","Map(vectorType -> sparse, length -> 32, indices -> List(5), values -> List(1.0))","Map(vectorType -> sparse, length -> 57, indices -> List(2, 3, 9, 21, 49, 53, 54), values -> List(1.0, 1.0, 1.0, 1.0, 3.0, 1.0, 56.0))","Map(vectorType -> dense, length -> 2, values -> List(7.150057315826416, -7.150057315826416))",0.0,"Map(vectorType -> dense, length -> 2, values -> List(0.9992157816886902, 7.842036429792643E-4))"
